<a href="https://colab.research.google.com/github/shanksghub/Minute-by-Minute-Price-prediction/blob/master/GRU%20reshape%20minute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:




#import all libraries
import numpy as np
import pandas as pd
import math
import sklearn
import sklearn.preprocessing
import datetime
import os
import matplotlib.pyplot as plt
import tensorflow as tf




url1 = 'https://raw.githubusercontent.com/shanksghub/Minute-by-Minute-Price-prediction/master/IBM.1Min.TradesOnly.20160128.csv'


# import dataset 
a = pd.read_csv(url1)



df.head()
a =df.drop(columns=['Ticker','TimeBarStart'])


In [215]:
a.head()

,Date,FirstTradePrice,HighTradePrice,LowTradePrice,LastTradePrice,VolumeWeightPrice,Volume,TotalTrades
0,20160128,121.99,121.99,121.95,121.95,121.97000,64,2
1,20160128,121.00,121.78,121.00,121.78,121.37818,132,5
2,20160128,121.00,121.00,120.88,120.88,120.92000,273,4
3,20160128,120.88,120.88,120.88,120.88,120.88000,500,4
4,20160128,120.88,120.88,120.88,120.88,120.88000,3,1


In [0]:
# data scaling (normalizing)
def normalize_data(a):
    min_max_scaler = sklearn.preprocessing.MinMaxScaler()
    a['FirstTradePrice'] = min_max_scaler.fit_transform(a.FirstTradePrice.values.reshape(-1,1))
    a['HighTradePrice'] = min_max_scaler.fit_transform(a.HighTradePrice.values.reshape(-1,1))
    a['LowTradePrice'] = min_max_scaler.fit_transform(a.LowTradePrice.values.reshape(-1,1))
    a['LastTradePrice'] = min_max_scaler.fit_transform(a['LastTradePrice'].values.reshape(-1,1))
    return a
a_norm = a.copy()
a_norm = normalize_data(a_norm)


In [0]:
# Splitting the dataset into Train, Valid & test data 
valid_set_size_percentage = 10 
test_set_size_percentage = 10 
seq_len = 20 # taken sequence length as 20
def load_data(a, seq_len):
    data_raw = a.as_matrix() 
    data = [] 
    for index in range(len(data_raw) - seq_len): 
        data.append(data_raw[index: index + seq_len])
    data = np.array(data);
    valid_set_size = int(np.round(valid_set_size_percentage/100*data.shape[0]));  
    test_set_size = int(np.round(test_set_size_percentage/100*data.shape[0]));
    train_set_size = data.shape[0] - (valid_set_size + test_set_size);
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    x_valid = data[train_set_size:train_set_size+valid_set_size,:-1,:]
    y_valid = data[train_set_size:train_set_size+valid_set_size,-1,:]
    x_test = data[train_set_size+valid_set_size:,:-1,:]
    y_test = data[train_set_size+valid_set_size:,-1,:]
    return [x_train, y_train, x_valid, y_valid, x_test, y_test]


In [218]:

x_train, y_train, x_valid, y_valid, x_test, y_test = load_data(a_norm, seq_len)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_valid.shape = ',x_valid.shape)
print('y_valid.shape = ', y_valid.shape)
print('x_test.shape = ', x_test.shape)
print('y_test.shape = ',y_test.shape)



x_train.shape =  (314, 19, 8)
y_train.shape =  (314, 8)
x_valid.shape =  (39, 19, 8)
y_valid.shape =  (39, 8)
x_test.shape =  (39, 19, 8)
y_test.shape =  (39, 8)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



In [0]:

n_steps = seq_len-1 
n_inputs = 4 
n_neurons = 20 
n_outputs = 4
n_layers = 2
learning_rate = 0.001
batch_size = 50
n_epochs = 10 
train_set_size = x_train.shape[0]
test_set_size = x_test.shape[0]
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_outputs])

In [0]:


index_in_epoch = 0;
perm_array  = np.arange(x_train.shape[0])
np.random.shuffle(perm_array)

def get_next_batch(batch_size):
    global index_in_epoch, x_train, perm_array   
    start = index_in_epoch
    index_in_epoch += batch_size 
    if index_in_epoch > x_train.shape[0]:
        np.random.shuffle(perm_array) # shuffle permutation array
        start = 0 # start next epoch
        index_in_epoch = batch_size     
    end = index_in_epoch
    return x_train[perm_array[start:end]], y_train[perm_array[start:end]]


In [0]:

#GRU 
layers = [tf.contrib.rnn.GRUCell(num_units=n_neurons, activation=tf.nn.relu)
          for layer in range(n_layers)] 

In [0]:
                                                                  
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
rnn_outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)
stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons]) 
stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs)
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])
outputs = outputs[:,n_steps-1,:]
                                              

In [0]:
# Cost function
loss = tf.reduce_mean(tf.square(outputs - y))

#optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) 
training_op = optimizer.minimize(loss)

In [224]:

                                              
# Fitting the model
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    for iteration in range(int(n_epochs*train_set_size/batch_size)):
        x_batch, y_batch = get_next_batch(batch_size) # fetch the next training batch 
        sess.run(training_op, feed_dict={X: x_batch, y: y_batch}) 
        if iteration % int(5*train_set_size/batch_size) == 0:
            mse_train = loss.eval(feed_dict={X: x_train, y: y_train}) 
            mse_valid = loss.eval(feed_dict={X: x_valid, y: y_valid}) 
            print('%.2f epochs: MSE train/valid = %.6f/%.6f'%(
                iteration*batch_size/train_set_size, mse_train, mse_valid))
            
            # Predictions
    y_test_pred = sess.run(outputs,feed_dict={X: x_test})
    

ValueError: ignored

In [213]:

    
#checking prediction output nos 
y_test_pred.shape


NameError: ignored

In [0]:

# ploting the graph
comp = pd.DataFrame({'Column1':y_test[:,3],'Column3':y_test_pred[:,3]})
plt.figure(figsize=(10,5))
plt.plot(comp['Column1'], color='blue', label='Target')
plt.plot(comp['Column3'], color='black', label='Prediction')
plt.legend()
plt.show()